In [6]:
import os

os.environ["DEEPSEEK_API_KEY"]='<Your Key>'
os.environ["LLM_MODEL"]='deepseek-chat'

# IaC：生成 Terraform 脚本

In [7]:
from langchain_openai.chat_models.base import BaseChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

# Deepseek api
llm = BaseChatOpenAI(
        model=os.getenv("LLM_MODEL"), 
        openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
        openai_api_base='https://api.deepseek.com',
        max_tokens=1024
    )
memory = ConversationBufferWindowMemory(k=10) 


def get_response(input):
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [8]:
print(get_response("""
给我一个terraform脚本示例，用来创建一个AWS EKS集群
"""))

当然！以下是一个使用Terraform创建AWS EKS集群的示例脚本。这个脚本会创建一个EKS集群、相关的VPC、子网、安全组、IAM角色等资源。

```hcl
provider "aws" {
  region = "us-west-2"
}

resource "aws_vpc" "eks_vpc" {
  cidr_block = "10.0.0.0/16"

  tags = {
    Name = "eks-vpc"
  }
}

resource "aws_subnet" "eks_subnet" {
  count = 2

  vpc_id            = aws_vpc.eks_vpc.id
  cidr_block        = cidrsubnet(aws_vpc.eks_vpc.cidr_block, 8, count.index)
  availability_zone = element(data.aws_availability_zones.available.names, count.index)

  tags = {
    Name = "eks-subnet-${count.index}"
  }
}

resource "aws_internet_gateway" "eks_igw" {
  vpc_id = aws_vpc.eks_vpc.id

  tags = {
    Name = "eks-igw"
  }
}

resource "aws_route_table" "eks_route_table" {
  vpc_id = aws_vpc.eks_vpc.id

  route {
    cidr_block = "0.0.0.0/0"
    gateway_id = aws_internet_gateway.eks_igw.id
  }

  tags = {
    Name = "eks-route-table"
  }
}

resource "aws_route_table_association" "eks_route_table_association" {
  count = 2

  subnet_id      = aws_subnet.eks_subnet[cou

In [9]:
print(get_response("""
将上面的script改写为适合阿里云的
"""))

当然！以下是将上述Terraform脚本改写为适合阿里云的版本。这个脚本会创建一个阿里云容器服务Kubernetes（ACK）集群、相关的VPC、子网、安全组、RAM角色等资源。

```hcl
provider "alicloud" {
  region = "cn-hangzhou"
}

resource "alicloud_vpc" "ack_vpc" {
  vpc_name   = "ack-vpc"
  cidr_block = "10.0.0.0/16"
}

resource "alicloud_vswitch" "ack_vswitch" {
  count = 2

  vpc_id            = alicloud_vpc.ack_vpc.id
  cidr_block        = cidrsubnet(alicloud_vpc.ack_vpc.cidr_block, 8, count.index)
  availability_zone = element(data.alicloud_zones.available.zones[*].id, count.index)
  vswitch_name      = "ack-vswitch-${count.index}"
}

resource "alicloud_nat_gateway" "ack_nat_gateway" {
  vpc_id = alicloud_vpc.ack_vpc.id
  name   = "ack-nat-gateway"
}

resource "alicloud_route_table" "ack_route_table" {
  vpc_id = alicloud_vpc.ack_vpc.id
  name   = "ack-route-table"
}

resource "alicloud_route_entry" "ack_route_entry" {
  count = 2

  route_table_id        = alicloud_route_table.ack_route_table.id
  destination_cidrblock = "0.0.0.0/0"
  nexthop_type          = "

# 生成应用部署文件

In [10]:
from openai import OpenAI

def work_on(input):

    client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat", 
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 1000,
      )
    return response.choices[0].message.content

In [11]:
print(work_on("""
编写kubernetes部署文件（yaml）用于部署Nginx服务，服务端口8080
满足以下的要求
1. 部署的namespace为”service“
2. 包含readiness及liveness probe
3. 采用一个独立的service account运行
4. 包含完美终止（graceful termination）配置
"""))

以下是一个满足你要求的 Kubernetes 部署文件（YAML），用于部署 Nginx 服务，服务端口为 8080：

```yaml
---
apiVersion: v1
kind: Namespace
metadata:
  name: service
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nginx-service-account
  namespace: service
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: service
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      serviceAccountName: nginx-service-account
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 8080
        livenessProbe:
          httpGet:
            path: /
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 10
        readinessProbe:
          httpGet:
            path: /
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 10
        lifecycle:
          preStop:
            exec:
              command: ["/bin

In [12]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain_openai.chat_models.base import BaseChatOpenAI

def create_k8s_config(requirements):
    prompt_template = """
    根据以下用户需求：
    "{requirements}"
    编写kubernetes部署文件（yaml), 
    满足以下的要求
    1. 部署的namespace为”service“
    2. 包含readiness及liveness probe
    3. 采用一个独立的service account运行
    4. 包含完美终止（graceful termination）配置
    """

    # Deepseek api
    llm = BaseChatOpenAI(
            model=os.getenv("LLM_MODEL"), 
            openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
            openai_api_base='https://api.deepseek.com',
            max_tokens=1024)
    
    llm_chain = LLMChain(
        llm=llm,
        prompt=PromptTemplate.from_template(prompt_template)
    )
    return llm_chain.run(requirements)

In [13]:
print(create_k8s_config("部署Nginx服务，服务端口8080"))

/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_24660/3945677529.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_24660/3945677529.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run(requirements)


以下是一个满足您需求的 Kubernetes 部署文件（YAML），用于部署 Nginx 服务，服务端口为 8080，并在 `service` 命名空间中运行。该部署文件包含了 `readiness` 和 `liveness` 探针、独立的 `ServiceAccount` 以及完美终止（graceful termination）配置。

```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: service
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nginx-service-account
  namespace: service
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      serviceAccountName: nginx-service-account
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 8080
        readinessProbe:
          httpGet:
            path: /
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 10
        livenessProbe:
          httpGet:
            path: /
            port: 8080
          initialDelaySeconds: 15
      